# Summary

Grab data from AnnData object and use it to create a graph pkl

## Background

Data can be represented in a more complete way by utilizing edge features in graph attention networks. 

## Objective

Use self-supervised learning to learn graphical representations of data and harness edge features in improving performance of predictive tasks

## Methods

GAT, edge features, self-supervised learning, representation learning, healthcare application, single-cell transcriptomic data

- val/test are equivalent here so we report performance on val set, since care only about what the network learns internally and use this for interpretability

## Results

## Conclusions

## Future directions

- use batch labels from dataset, train GAT to get edge coefficients from preds of those labels, use this for "batch effect correction" in the model, either by penalizing reliance on these edge features, or controlling for them in the final model 
- 

In [1]:
%matplotlib inline
%load_ext memory_profiler

import os
import time
import datetime
import sys
import pickle
import scanpy as sc
from scipy import sparse
import numpy as np
from sklearn.model_selection import train_test_split

sc.settings.verbosity=2

#### Load AnnData obj

In [2]:
# fps
pfp = '/home/ngr4/project/scnd/results/'
pdfp = '/home/ngr4/project/scnd/data/processed/'

def loader(fname,fpath,backed=None) : 
    start = time.time()
    adata = sc.read_h5ad(filename=os.path.join(fpath,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    return adata

def writer(fname,fpath,AnnData) :
    start = time.time()
    Anndata.write(os.path.join(fpath,fname))
    print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to save data'.format(time.time()-start))
    

if False :
    # load human
    fname='hum_MT_bbknn.h5ad'
    %memit hdata = loader(fname,pdfp)
    
if True :
    # load mouse
    fname='mouse_MT_bbknn.h5ad'
    %memit adata = loader(fname,pdfp)

loaded @200528.22:25:09
took 79.43-s to load data
peak memory: 74841.77 MiB, increment: 74658.86 MiB


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/anndata/compat/__init__.py:161: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/anndata/compat/__init__.py:161: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  FutureWarning,


# Induction

Sample 1/3 of the data randomly, grab labels

In [14]:
# sample the data 
idx_train, idx_test = train_test_split(adata.obs.index, train_size=0.33)
%memit tdata = sc.AnnData(X=adata[adata.obs.index.isin(idx_train),:].X, obs=adata[adata.obs.index.isin(idx_train),:].obs)
temp = adata.obs.index[adata.obs.index.isin(idx_test)].to_list()
idx_val, idx_test = train_test_split(temp, train_size=0.1)
val = sc.AnnData(X=adata[adata.obs.index.isin(idx_val),:].X, obs=adata[adata.obs.index.isin(idx_val),:].obs)
temp = adata.obs.index[adata.obs.index.isin(idx_test)].to_list()
idx_test, _ = train_test_split(temp, train_size=0.11)
test = sc.AnnData(X=adata[adata.obs.index.isin(idx_test),:].X, obs=adata[adata.obs.index.isin(idx_test),:].obs)

def graph_pp(AnnData, bbknn=True):
    sc.tl.pca(AnnData, n_comps=50)
    if bbknn:
        sc.external.pp.bbknn(AnnData)
    else:
        sc.pp.neighbors(AnnData, n_pcs=100, n_neighbors=30)
    return AnnData

# make graph
tdata = graph_pp(tdata)
val = graph_pp(val)
test = graph_pp(test)

if False:
    del adata

peak memory: 83275.46 MiB, increment: 4650.95 MiB
computing PCA
    with n_comps=50
    finished (0:02:55)
computing batch balanced neighbors
	finished (0:02:27)
computing PCA
    with n_comps=50


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:294: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['distances'] = bbknn_out[0]
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:295: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'] = bbknn_out[1]


    finished (0:01:11)
computing batch balanced neighbors
	finished (0:00:25)
computing PCA
    with n_comps=50


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:294: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['distances'] = bbknn_out[0]
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:295: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'] = bbknn_out[1]


    finished (0:01:14)
computing batch balanced neighbors
	finished (0:00:25)


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:294: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['distances'] = bbknn_out[0]
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:295: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'] = bbknn_out[1]


## Encoding 

Select tasks for prediction

1. yctype
2. ysca1
3. ygenotime (already done)
4. SCA1_5/12/18/24/30wk


In [16]:
# encode ctype 
ctype_encoder = {v:i for i,v in enumerate(tdata.obs['ctype'].unique())}
tdata.obs['yctype'] = tdata.obs['ctype'].map(ctype_encoder)
val.obs['yctype'] = val.obs['ctype'].map(ctype_encoder)
test.obs['yctype'] = test.obs['ctype'].map(ctype_encoder)

# encode WT/SCA1 blended across time
genotype_encoder = {'WT':0, 'SCA1':1}
tdata.obs['ysca1'] = tdata.obs['genotype'].map(genotype_encoder).astype(int)
val.obs['ysca1'] = val.obs['genotype'].map(genotype_encoder).astype(int)
test.obs['ysca1'] = test.obs['genotype'].map(genotype_encoder).astype(int)

# encode multi-label
tdata.obs['genotype_timepoint'] = tdata.obs['genotype'].astype(str) + tdata.obs['timepoint'].astype(str).apply(lambda x: '_{}'.format(x))
val.obs['genotype_timepoint'] = val.obs['genotype'].astype(str) + val.obs['timepoint'].astype(str).apply(lambda x: '_{}'.format(x))
test.obs['genotype_timepoint'] = test.obs['genotype'].astype(str) + test.obs['timepoint'].astype(str).apply(lambda x: '_{}'.format(x))

gt_encoder = {v:i for i,v in enumerate(tdata.obs['genotype_timepoint'].unique())}
tdata.obs['ygenotime'] = tdata.obs['genotype_timepoint'].map(gt_encoder)
val.obs['ygenotime'] = val.obs['genotype_timepoint'].map(gt_encoder)
test.obs['ygenotime'] = test.obs['genotype_timepoint'].map(gt_encoder)

# encode distinguishability of SCA1 at specific timepoints 
verbose = False
tdata.obs['SCA1_5wk'] = (tdata.obs['genotype_timepoint']=='SCA1_5wk').astype(int)
tdata.obs['SCA1_12wk'] = (tdata.obs['genotype_timepoint']=='SCA1_12wk').astype(int)
tdata.obs['SCA1_18wk'] = (tdata.obs['genotype_timepoint']=='SCA1_18wk').astype(int)
tdata.obs['SCA1_24wk'] = (tdata.obs['genotype_timepoint']=='SCA1_24wk').astype(int)
tdata.obs['SCA1_30wk'] = (tdata.obs['genotype_timepoint']=='SCA1_30wk').astype(int)
val.obs['SCA1_5wk'] = (val.obs['genotype_timepoint']=='SCA1_5wk').astype(int)
val.obs['SCA1_12wk'] = (val.obs['genotype_timepoint']=='SCA1_12wk').astype(int)
val.obs['SCA1_18wk'] = (val.obs['genotype_timepoint']=='SCA1_18wk').astype(int)
val.obs['SCA1_24wk'] = (val.obs['genotype_timepoint']=='SCA1_24wk').astype(int)
val.obs['SCA1_30wk'] = (val.obs['genotype_timepoint']=='SCA1_30wk').astype(int)
test.obs['SCA1_5wk'] = (test.obs['genotype_timepoint']=='SCA1_5wk').astype(int)
test.obs['SCA1_12wk'] = (test.obs['genotype_timepoint']=='SCA1_12wk').astype(int)
test.obs['SCA1_18wk'] = (test.obs['genotype_timepoint']=='SCA1_18wk').astype(int)
test.obs['SCA1_24wk'] = (test.obs['genotype_timepoint']=='SCA1_24wk').astype(int)
test.obs['SCA1_30wk'] = (test.obs['genotype_timepoint']=='SCA1_30wk').astype(int)

verbose = False
if verbose:
    # check encoding 
    print(tdata.obs['genotype_timepoint'].value_counts())
    for i in ['SCA1_5wk', 'SCA1_12wk','SCA1_18wk','SCA1_24wk','SCA1_30wk']:
        print(tdata.obs[i].sum())
        
    print(test.obs['genotype_timepoint'].value_counts())
    for i in ['SCA1_5wk', 'SCA1_12wk','SCA1_18wk','SCA1_24wk','SCA1_30wk']:
        print(test.obs[i].sum())



In [18]:
# create dictionary
def dictthat(AnnData, gene_ranger=True):
    """Prep dictionary for export.
    
    If gene_ranger, divide by zero can occur for 
    non-expressing genes. Thus, will floor those
    to 0.
    
    NOTE: customization re:y to predict is highly
    dependent on user input. ERGO, modify this 
    
    Arguments:
        AnnData (sc.AnnData): with graph stuff
        
    Returns:
        dict
    """
    if gene_ranger:
        # each gene in [0,1], divide by zeros to 0
        minimum = AnnData.X.min(axis=0)
        maximum = AnnData.X.max(axis=0)
        num = AnnData.X - minimum.todense()
        denom =  (maximum - minimum).todense()
        xhat = np.divide(num, denom, out=np.zeros_like(num), where=denom!=0) 
    else:
        # matrix in [0,1]
        xhat = (AnnData.X - AnnData.X.min()) / (AnnData.X.max() - AnnData.X.min())
        
    

    gdata = {'X':xhat,
             'adj':AnnData.uns['neighbors']['connectivities']+sparse.diags([1]*AnnData.shape[0], format='csr'),
             'feature_names':AnnData.var_names.to_list()}
    gdata['cell_id'] = AnnData.obs.index.to_list()
    for col in AnnData.obs.columns:
        gdata[col] = AnnData.obs[col].to_list()
    
    return gdata

gdata_train = dictthat(tdata)
gdata_val = dictthat(val)
gdata_test  = dictthat(test)



/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.


In [19]:
# export
def pklthat(gdata, fname, fpath=pdfp): 
    with open(os.path.join(fpath,fname),'wb') as f :
        pickle.dump(gdata, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()

pklthat(gdata_train, 'scnd_train_200528.pkl')
pklthat(gdata_val, 'scnd_val_200528.pkl')
pklthat(gdata_test, 'scnd_test_200528.pkl')

# clean
if True:
    del tdata, test, gdata_train, gdata_test

## modify pkl

Add batch encoding

In [50]:
def loadpkl(filename):
    with open (filename, 'rb') as f:
        temp = pickle.load(f)
        f.close()
    return temp

def add_batch(filename, date='200529'):
    gdata = loadpkl(filename)
    batch_encoder = {v:i for i,v in enumerate(np.unique(gdata['batch']))}
    gdata['ybatch'] = list(map(batch_encoder.get, gdata['batch']))
    pklthat(gdata, '{}_{}.pkl'.format(os.path.split(filename)[1].split('_20')[0], date))
    del gdata
    print('Batch added and pkl saved:\n  {}_{}.pkl'.format(os.path.split(filename)[1].split('_20')[0], date))

In [51]:
add_batch(os.path.join(pdfp,'scnd_train_200528.pkl'))
add_batch(os.path.join(pdfp,'scnd_val_200528.pkl'))
add_batch(os.path.join(pdfp,'scnd_test_200528.pkl'))

Batch added and pkl saved:
  scnd_train.pkl
Batch added and pkl saved:
  scnd_val.pkl
Batch added and pkl saved:
  scnd_test.pkl


In [53]:
# test
gdata = loadpkl(os.path.join(pdfp,'scnd_train_200529.pkl'))